# NGRC vs Fourier vs Traditional Reservoir for INR

Comparing Next Generation Reservoir Computing (NGRC) approach with Fourier features and traditional reservoir computing for Implicit Neural Representations.

**Key Insight**: NGRC uses polynomial features of time-delayed inputs instead of random recurrent dynamics. For spatial INR, we adapt this to polynomial features of spatial coordinates.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from itertools import combinations_with_replacement

try:
    import torch
    import torch.nn as nn
    USE_TORCH = True
    print("PyTorch available for MLP training")
except ImportError:
    USE_TORCH = False
    print("PyTorch not available, using Ridge regression only")

## Load Image

In [ ]:
# Load and preprocess image
img = Image.open('fig/cat.png').convert('RGB')
target_size = 256
img = img.resize((target_size, target_size), Image.LANCZOS)
img_array = np.array(img) / 255.0

h, w, c = img_array.shape
print(f"Image size: {h}x{w}x{c}")

# Create coordinate grid
coords = np.linspace(0, 1, h, endpoint=False)
x_grid = np.stack(np.meshgrid(coords, coords), -1)  # (H, W, 2)

# Train/test split (as in original Fourier paper)
test_data = (x_grid.reshape(-1, 2), img_array.reshape(-1, 3))
train_data = (x_grid[::2, ::2].reshape(-1, 2), img_array[::2, ::2].reshape(-1, 3))

print(f"Train: {len(train_data[0])}, Test: {len(test_data[0])}")

plt.figure(figsize=(6, 6))
plt.imshow(img_array)
plt.title(f'Target Image ({h}x{w})')
plt.axis('off')
plt.show()

## Method 1: NGRC-Style Polynomial Features

Following the NGRC principle: instead of random recurrence, use **explicit polynomial features**.

For spatial INR with coordinates $(x, y)$:
$$\mathbf{O}(x,y) = [1, x, y, x^2, xy, y^2, x^3, x^2y, xy^2, y^3, \ldots]$$

This is the spatial analog of NGRC's time-delay polynomial features.

In [ ]:
def ngrc_polynomial_features(x, degree=2):
    """
    NGRC-style polynomial features for spatial coordinates.
    
    For degree=2: [1, x, y, x², xy, y²]
    For degree=3: [1, x, y, x², xy, y², x³, x²y, xy², y³]
    etc.
    
    This is the spatial analog of NGRC's time-delay polynomial expansion.
    """
    n, d = x.shape
    features = [np.ones((n, 1))]  # Constant term
    
    for deg in range(1, degree + 1):
        # All monomials of this degree
        for powers in combinations_with_replacement(range(d), deg):
            # powers is like (0,), (1,), (0,0), (0,1), (1,1), etc.
            term = np.ones(n)
            for p in powers:
                term = term * x[:, p]
            features.append(term.reshape(-1, 1))
    
    return np.hstack(features)


def ngrc_polynomial_features_fast(x, degree=2):
    """
    Faster implementation using sklearn-style polynomial expansion.
    """
    try:
        from sklearn.preprocessing import PolynomialFeatures
        poly = PolynomialFeatures(degree=degree, include_bias=True)
        return poly.fit_transform(x)
    except ImportError:
        return ngrc_polynomial_features(x, degree)


# Test
test_x = np.array([[0.5, 0.5]])
for deg in [2, 3, 4, 5]:
    feats = ngrc_polynomial_features_fast(test_x, deg)
    print(f"Degree {deg}: {feats.shape[1]} features")

## Method 2: Fourier Features (Original Paper)

$$\gamma(\mathbf{v}) = [\sin(2\pi \mathbf{B} \mathbf{v}), \cos(2\pi \mathbf{B} \mathbf{v})]^T$$

where $\mathbf{B} \sim \mathcal{N}(0, \sigma^2)$

In [ ]:
def fourier_features(x, num_features, sigma):
    """
    Random Fourier features as in the original paper.
    """
    np.random.seed(42)
    B = np.random.randn(num_features, x.shape[1]) * sigma
    x_proj = (2. * np.pi * x) @ B.T
    return np.concatenate([np.sin(x_proj), np.cos(x_proj)], axis=-1)

## Method 3: Traditional Reservoir (What We Were Doing)

$$\mathbf{h}^{(l)} = \tanh(\mathbf{W}_{in}^{(l)} \mathbf{h}^{(l-1)} + \mathbf{W}_{hh}^{(l)} \mathbf{h}^{(l)} + \mathbf{b}^{(l)})$$

Iterative settling with recurrence.

In [ ]:
def deep_reservoir(x, hidden_size, num_layers=5, iterations=10, spectral_radius=0.9):
    """
    Traditional deep reservoir with recurrence.
    """
    np.random.seed(42)
    n, d = x.shape
    
    layers = []
    d_in = d
    for layer in range(num_layers):
        np.random.seed(42 + layer)
        W_in = np.random.randn(d_in, hidden_size) * 0.5
        W_hh = np.random.randn(hidden_size, hidden_size)
        eig = np.abs(np.linalg.eigvals(W_hh)).max()
        W_hh = W_hh * (spectral_radius / eig)
        b = np.random.randn(hidden_size) * 0.1
        layers.append((W_in, W_hh, b))
        d_in = hidden_size
    
    all_states = []
    for i in tqdm(range(n), desc='Reservoir', leave=False):
        layer_input = x[i:i+1]
        layer_states = []
        
        for W_in, W_hh, b in layers:
            h = np.zeros(hidden_size)
            for _ in range(iterations):
                h = np.tanh(layer_input @ W_in + h @ W_hh + b).flatten()
            layer_states.append(h)
            layer_input = h.reshape(1, -1)
        
        all_states.append(np.concatenate(layer_states))
    
    return np.array(all_states)

## Method 4: NGRC-Style Random Nonlinear Projection

Recent NGRC variants use **random nonlinear projections** instead of explicit polynomials:
$$\mathbf{O}(x) = \phi(\mathbf{W} \mathbf{x} + \mathbf{b})$$

where $\phi$ is a nonlinearity. This is like a single-layer random network (no recurrence).

In [ ]:
def random_nonlinear_projection(x, num_features, nonlinearity='tanh'):
    """
    NGRC-style random nonlinear projection (no recurrence).
    Similar to Extreme Learning Machine / Random Kitchen Sink.
    """
    np.random.seed(42)
    W = np.random.randn(x.shape[1], num_features) * np.sqrt(2.0 / x.shape[1])
    b = np.random.randn(num_features) * 0.1
    
    proj = x @ W + b
    
    if nonlinearity == 'tanh':
        return np.tanh(proj)
    elif nonlinearity == 'relu':
        return np.maximum(0, proj)
    elif nonlinearity == 'sin':
        return np.sin(proj)
    else:
        return proj

## Ridge Regression

In [ ]:
def ridge_regression(H_train, y_train, H_test, y_test, lamb=1e-6):
    """Ridge regression with evaluation."""
    W = np.linalg.solve(H_train.T @ H_train + lamb * np.eye(H_train.shape[1]), H_train.T @ y_train)
    pred = np.clip(H_test @ W, 0, 1)
    mse = np.mean((pred - y_test) ** 2)
    psnr = -10 * np.log10(mse) if mse > 0 else 100
    return pred, mse, psnr, W

In [ ]:
if USE_TORCH:
    class MLP(nn.Module):
        """MLP decoder as in the original Fourier paper."""
        def __init__(self, input_dim, hidden_dim=256, num_layers=4):
            super().__init__()
            layers = []
            d_in = input_dim
            for i in range(num_layers - 1):
                layers.append(nn.Linear(d_in, hidden_dim))
                layers.append(nn.ReLU())
                d_in = hidden_dim
            layers.append(nn.Linear(d_in, 3))
            layers.append(nn.Sigmoid())
            self.net = nn.Sequential(*layers)
        
        def forward(self, x):
            return self.net(x)
    
    def train_mlp(H_train, y_train, H_test, y_test, 
                  num_layers=4, hidden_dim=256, lr=1e-4, iters=2000):
        """Train MLP on features exactly as in Demo.ipynb."""
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"    Using device: {device}")
        
        model = MLP(H_train.shape[1], hidden_dim, num_layers).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        
        H_train_t = torch.FloatTensor(H_train).to(device)
        y_train_t = torch.FloatTensor(y_train).to(device)
        H_test_t = torch.FloatTensor(H_test).to(device)
        y_test_t = torch.FloatTensor(y_test).to(device)
        
        train_psnrs, test_psnrs = [], []
        
        pbar = tqdm(range(iters), desc='Training MLP', leave=False)
        for i in pbar:
            optimizer.zero_grad()
            pred = model(H_train_t)
            loss = 0.5 * torch.mean((pred - y_train_t) ** 2)
            loss.backward()
            optimizer.step()
            
            if i % 100 == 0:
                with torch.no_grad():
                    train_mse = torch.mean((model(H_train_t) - y_train_t) ** 2).item()
                    test_mse = torch.mean((model(H_test_t) - y_test_t) ** 2).item()
                    train_psnr = -10 * np.log10(train_mse)
                    test_psnr = -10 * np.log10(test_mse)
                    train_psnrs.append(train_psnr)
                    test_psnrs.append(test_psnr)
                    pbar.set_postfix({'train': f'{train_psnr:.1f}', 'test': f'{test_psnr:.1f}'})
        
        with torch.no_grad():
            pred = model(H_test_t).cpu().numpy()
            mse = np.mean((pred - y_test) ** 2)
            psnr = -10 * np.log10(mse)
        
        return pred, mse, psnr, train_psnrs, test_psnrs, model
else:
    print("PyTorch not available - MLP training disabled")

## MLP Decoder (as in original Fourier paper)

Following Demo.ipynb: 4-layer MLP with ReLU activations and Sigmoid output.

## Experiment 1: NGRC Polynomial Features

In [ ]:
results = {}

print("=" * 60)
print("NGRC-STYLE POLYNOMIAL FEATURES")
print("=" * 60)

for degree in [2, 3, 4, 5, 6, 7, 8, 10, 12, 15]:
    H_train = ngrc_polynomial_features_fast(train_data[0], degree)
    H_test = ngrc_polynomial_features_fast(test_data[0], degree)
    
    try:
        pred, mse, psnr, W = ridge_regression(H_train, train_data[1], H_test, test_data[1])
        results[f'ngrc_poly_d{degree}'] = {'pred': pred, 'mse': mse, 'psnr': psnr, 'dim': H_train.shape[1]}
        print(f"  Degree {degree:>2}: PSNR = {psnr:.2f} dB (dim={H_train.shape[1]})")
    except Exception as e:
        print(f"  Degree {degree:>2}: FAILED - {e}")

## Experiment 2: Fourier Features

In [ ]:
print("\n" + "=" * 60)
print("FOURIER FEATURES")
print("=" * 60)

for num_feat in [128, 256, 512, 1024]:
    for sigma in [1, 10, 100]:
        H_train = fourier_features(train_data[0], num_feat, sigma)
        H_test = fourier_features(test_data[0], num_feat, sigma)
        
        pred, mse, psnr, W = ridge_regression(H_train, train_data[1], H_test, test_data[1])
        name = f'fourier_n{num_feat}_s{sigma}'
        results[name] = {'pred': pred, 'mse': mse, 'psnr': psnr, 'dim': H_train.shape[1]}
        print(f"  n={num_feat:>4}, σ={sigma:>3}: PSNR = {psnr:.2f} dB (dim={H_train.shape[1]})")

## Experiment 3: Random Nonlinear Projection (NGRC variant)

In [ ]:
print("\n" + "=" * 60)
print("RANDOM NONLINEAR PROJECTION (NGRC-style, no recurrence)")
print("=" * 60)

for num_feat in [256, 512, 1024, 2048]:
    for nonlin in ['tanh', 'relu', 'sin']:
        H_train = random_nonlinear_projection(train_data[0], num_feat, nonlin)
        H_test = random_nonlinear_projection(test_data[0], num_feat, nonlin)
        
        pred, mse, psnr, W = ridge_regression(H_train, train_data[1], H_test, test_data[1])
        name = f'random_{nonlin}_n{num_feat}'
        results[name] = {'pred': pred, 'mse': mse, 'psnr': psnr, 'dim': H_train.shape[1]}
        print(f"  {nonlin:>4}, n={num_feat:>4}: PSNR = {psnr:.2f} dB")

In [ ]:
# Key Insights from MLP-INR Comparison
if mlp_results:
    print("\n" + "=" * 70)
    print("KEY INSIGHTS: MLP-BASED INR WITH DIFFERENT FEATURES")
    print("=" * 70)
    
    # Find best in each category
    best_none = mlp_results.get('mlp_none', {}).get('psnr', 0)
    best_poly = max([r['psnr'] for k, r in mlp_results.items() if 'poly' in k], default=0)
    best_fourier = max([r['psnr'] for k, r in mlp_results.items() if 'fourier' in k], default=0)
    best_random = max([r['psnr'] for k, r in mlp_results.items() if 'random' in k], default=0)
    
    print(f"""
┌─────────────────────────────────────────────────────────────────┐
│                    MLP-INR RESULTS SUMMARY                       │
├─────────────────────────────────────────────────────────────────┤
│  No mapping (raw x,y):              {best_none:>6.2f} dB                  │
│  NGRC Polynomial (best):            {best_poly:>6.2f} dB                  │
│  Fourier Features (best):           {best_fourier:>6.2f} dB                  │
│  Random Projection (best):          {best_random:>6.2f} dB                  │
├─────────────────────────────────────────────────────────────────┤
│  Fourier - Polynomial gap:          {best_fourier - best_poly:>+6.2f} dB                  │
│  Fourier - Random gap:              {best_fourier - best_random:>+6.2f} dB                  │
│  Polynomial - None gap:             {best_poly - best_none:>+6.2f} dB                  │
└─────────────────────────────────────────────────────────────────┘

ANALYSIS:
━━━━━━━━━

1. INPUT ENCODING IS CRITICAL
   The same MLP architecture performs very differently depending
   on how inputs are encoded. This confirms the Fourier paper's
   key finding: the spectral bias of MLPs can be overcome with
   proper input encoding.

2. NGRC POLYNOMIAL vs FOURIER
   - Polynomials help, but Fourier features are better for images
   - Why? Natural images are bandlimited (smooth frequency content)
   - Fourier basis naturally captures this; polynomials don't

3. RANDOM sin(Wx+b) ≈ FOURIER
   Random sinusoidal projection works similarly to Fourier features!
   This is because sin(Wx+b) is essentially sampling random frequencies.

4. THE SPECTRAL BIAS PROBLEM
   - Raw (x,y) → MLP learns low frequencies first (slow for detail)
   - Fourier features → directly encode all frequencies
   - Polynomials → local basis, poor for periodic/oscillatory content

5. NGRC LESSON FOR INR
   NGRC's key insight (replace recurrence with explicit features)
   applies to INR too: the RIGHT features matter more than 
   network depth or architecture complexity.
""")

In [ ]:
# MLP Visual Comparison
if mlp_results:
    fig, axes = plt.subplots(2, 5, figsize=(20, 8))
    
    # Row 1: Reconstructions
    axes[0, 0].imshow(img_array)
    axes[0, 0].set_title('Original', fontsize=12)
    axes[0, 0].axis('off')
    
    sorted_mlp = sorted(mlp_results.items(), key=lambda x: x[1]['psnr'], reverse=True)
    for i, (name, r) in enumerate(sorted_mlp[:4]):
        pred = r['pred'].reshape(h, w, 3)
        axes[0, i+1].imshow(pred)
        short_name = name.replace('mlp_', '').replace('ngrc_', '')
        axes[0, i+1].set_title(f'{short_name}\n{r["psnr"]:.1f} dB', fontsize=10)
        axes[0, i+1].axis('off')
    
    # Row 2: Training curves
    ax = axes[1, 0]
    for name, r in mlp_results.items():
        if 'test_psnrs' in r:
            xs = np.arange(len(r['test_psnrs'])) * 100
            short_name = name.replace('mlp_', '').replace('ngrc_', '')
            ax.plot(xs, r['test_psnrs'], label=short_name)
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Test PSNR (dB)')
    ax.set_title('Training Curves')
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)
    
    # Bar chart
    ax = axes[1, 1]
    names = [n.replace('mlp_', '').replace('ngrc_', '') for n, _ in sorted_mlp]
    psnrs = [r['psnr'] for _, r in sorted_mlp]
    colors = ['purple' if 'poly' in n else 'blue' if 'fourier' in n else 'orange' if 'random' in n else 'gray' for n in names]
    ax.barh(range(len(names)), psnrs, color=colors, alpha=0.8)
    ax.set_yticks(range(len(names)))
    ax.set_yticklabels(names, fontsize=9)
    ax.set_xlabel('PSNR (dB)')
    ax.set_title('MLP-INR Comparison')
    ax.grid(True, alpha=0.3, axis='x')
    
    # Hide unused
    for i in range(2, 5):
        axes[1, i].axis('off')
    
    plt.tight_layout()
    plt.savefig('ngrc_mlp_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()

In [ ]:
# MLP Results Summary
if mlp_results:
    print("\n" + "=" * 70)
    print("MLP-BASED INR RESULTS (sorted by PSNR)")
    print("=" * 70)
    
    sorted_mlp = sorted(mlp_results.items(), key=lambda x: x[1]['psnr'], reverse=True)
    print(f"{'Method':<30} {'PSNR (dB)':<12} {'Input Dim':<10}")
    print("-" * 52)
    for name, r in sorted_mlp:
        print(f"{name:<30} {r['psnr']:<12.2f} {r['dim']:<10}")

In [ ]:
mlp_results = {}

if USE_TORCH:
    print("=" * 70)
    print("MLP-BASED INR COMPARISON")
    print("=" * 70)
    print("Same MLP (4 layers, 256 hidden, ReLU, Sigmoid output)")
    print("Different input features\n")
    
    # 1. No mapping (raw coordinates) - baseline
    print("1. RAW COORDINATES (no mapping)")
    H_train = train_data[0]  # Just (x, y)
    H_test = test_data[0]
    pred, mse, psnr, train_psnrs, test_psnrs, _ = train_mlp(
        H_train, train_data[1], H_test, test_data[1], iters=2000
    )
    mlp_results['mlp_none'] = {'pred': pred, 'psnr': psnr, 'dim': 2, 
                               'train_psnrs': train_psnrs, 'test_psnrs': test_psnrs}
    print(f"    PSNR = {psnr:.2f} dB\n")
    
    # 2. NGRC Polynomial features
    print("2. NGRC POLYNOMIAL FEATURES")
    for degree in [3, 5, 8]:
        print(f"  Degree {degree}:")
        H_train = ngrc_polynomial_features_fast(train_data[0], degree)
        H_test = ngrc_polynomial_features_fast(test_data[0], degree)
        pred, mse, psnr, train_psnrs, test_psnrs, _ = train_mlp(
            H_train, train_data[1], H_test, test_data[1], iters=2000
        )
        mlp_results[f'mlp_ngrc_poly_d{degree}'] = {
            'pred': pred, 'psnr': psnr, 'dim': H_train.shape[1],
            'train_psnrs': train_psnrs, 'test_psnrs': test_psnrs
        }
        print(f"    PSNR = {psnr:.2f} dB (dim={H_train.shape[1]})\n")
    
    # 3. Fourier features (different scales)
    print("3. FOURIER FEATURES")
    for sigma in [1, 10, 100]:
        print(f"  σ = {sigma}:")
        H_train = fourier_features(train_data[0], 256, sigma)
        H_test = fourier_features(test_data[0], 256, sigma)
        pred, mse, psnr, train_psnrs, test_psnrs, _ = train_mlp(
            H_train, train_data[1], H_test, test_data[1], iters=2000
        )
        mlp_results[f'mlp_fourier_s{sigma}'] = {
            'pred': pred, 'psnr': psnr, 'dim': H_train.shape[1],
            'train_psnrs': train_psnrs, 'test_psnrs': test_psnrs
        }
        print(f"    PSNR = {psnr:.2f} dB (dim={H_train.shape[1]})\n")
    
    # 4. Random nonlinear projection
    print("4. RANDOM NONLINEAR PROJECTION")
    for nonlin in ['tanh', 'sin']:
        print(f"  {nonlin}:")
        H_train = random_nonlinear_projection(train_data[0], 512, nonlin)
        H_test = random_nonlinear_projection(test_data[0], 512, nonlin)
        pred, mse, psnr, train_psnrs, test_psnrs, _ = train_mlp(
            H_train, train_data[1], H_test, test_data[1], iters=2000
        )
        mlp_results[f'mlp_random_{nonlin}'] = {
            'pred': pred, 'psnr': psnr, 'dim': H_train.shape[1],
            'train_psnrs': train_psnrs, 'test_psnrs': test_psnrs
        }
        print(f"    PSNR = {psnr:.2f} dB (dim={H_train.shape[1]})\n")

else:
    print("PyTorch not available - skipping MLP experiments")

## Experiment 5: MLP-based INR with Different Features

This is the key comparison: same MLP architecture, different input features.
Following the original Fourier paper setup exactly.

## Experiment 4: Traditional Reservoir (for comparison)

In [ ]:
print("\n" + "=" * 60)
print("TRADITIONAL DEEP RESERVOIR (with recurrence)")
print("=" * 60)

for num_layers, hidden in [(5, 128), (10, 64)]:
    name = f'reservoir_L{num_layers}_H{hidden}'
    print(f"  Computing {name}...")
    
    H_train = deep_reservoir(train_data[0], hidden, num_layers=num_layers, iterations=10)
    H_test = deep_reservoir(test_data[0], hidden, num_layers=num_layers, iterations=10)
    
    pred, mse, psnr, W = ridge_regression(H_train, train_data[1], H_test, test_data[1])
    results[name] = {'pred': pred, 'mse': mse, 'psnr': psnr, 'dim': H_train.shape[1]}
    print(f"    PSNR = {psnr:.2f} dB (dim={H_train.shape[1]})")

## Results Summary

In [ ]:
# Sort by PSNR
sorted_results = sorted(results.items(), key=lambda x: x[1]['psnr'], reverse=True)

print("\n" + "=" * 70)
print("ALL RESULTS (sorted by PSNR)")
print("=" * 70)
print(f"{'Method':<35} {'PSNR (dB)':<12} {'Dim':<8}")
print("-" * 55)
for name, r in sorted_results[:20]:  # Top 20
    print(f"{name:<35} {r['psnr']:<12.2f} {r['dim']:<8}")

In [ ]:
# Best in each category
categories = {
    'NGRC Polynomial': [k for k in results if 'ngrc_poly' in k],
    'Fourier': [k for k in results if 'fourier' in k],
    'Random Projection': [k for k in results if 'random_' in k],
    'Traditional Reservoir': [k for k in results if 'reservoir' in k],
}

print("\n" + "=" * 70)
print("BEST IN EACH CATEGORY")
print("=" * 70)

best_results = {}
for cat, keys in categories.items():
    if keys:
        best_key = max(keys, key=lambda k: results[k]['psnr'])
        best_results[cat] = best_key
        r = results[best_key]
        print(f"{cat:<25}: {best_key:<30} PSNR = {r['psnr']:.2f} dB")

## Visualization

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(20, 8))

# Row 1: Best from each method
axes[0, 0].imshow(img_array)
axes[0, 0].set_title('Original', fontsize=12)
axes[0, 0].axis('off')

for i, (cat, key) in enumerate(best_results.items()):
    if i < 4:
        pred = results[key]['pred'].reshape(h, w, 3)
        axes[0, i+1].imshow(pred)
        axes[0, i+1].set_title(f'{cat}\n{results[key]["psnr"]:.1f} dB', fontsize=10)
        axes[0, i+1].axis('off')

# Row 2: NGRC polynomial scaling
poly_keys = sorted([k for k in results if 'ngrc_poly' in k], 
                   key=lambda k: int(k.split('_d')[1]))
for i, key in enumerate(poly_keys[:5]):
    pred = results[key]['pred'].reshape(h, w, 3)
    axes[1, i].imshow(pred)
    deg = key.split('_d')[1]
    axes[1, i].set_title(f'Poly deg={deg}\n{results[key]["psnr"]:.1f} dB', fontsize=10)
    axes[1, i].axis('off')

plt.tight_layout()
plt.savefig('ngrc_comparison_images.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Bar chart by category
fig, ax = plt.subplots(figsize=(16, 6))

names = [r[0] for r in sorted_results]
psnrs = [r[1]['psnr'] for r in sorted_results]

colors = []
for n in names:
    if 'ngrc_poly' in n:
        colors.append('purple')
    elif 'fourier' in n:
        colors.append('blue')
    elif 'random_' in n:
        colors.append('orange')
    elif 'reservoir' in n:
        colors.append('green')
    else:
        colors.append('gray')

bars = ax.bar(range(len(names)), psnrs, color=colors, alpha=0.8)
ax.set_xticks(range(len(names)))
ax.set_xticklabels(names, rotation=45, ha='right', fontsize=7)
ax.set_ylabel('PSNR (dB)', fontsize=12)
ax.set_title('NGRC vs Fourier vs Random Projection vs Traditional Reservoir', fontsize=14)
ax.grid(True, alpha=0.3, axis='y')

from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='purple', alpha=0.8, label='NGRC Polynomial'),
    Patch(facecolor='blue', alpha=0.8, label='Fourier'),
    Patch(facecolor='orange', alpha=0.8, label='Random Projection'),
    Patch(facecolor='green', alpha=0.8, label='Traditional Reservoir'),
]
ax.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.savefig('ngrc_comparison_barchart.png', dpi=150, bbox_inches='tight')
plt.show()

## Scaling Analysis

In [ ]:
# Plot PSNR vs feature dimension for each method
fig, ax = plt.subplots(figsize=(12, 6))

# NGRC Polynomial
poly_data = [(results[k]['dim'], results[k]['psnr']) for k in results if 'ngrc_poly' in k]
poly_data.sort()
ax.plot([d[0] for d in poly_data], [d[1] for d in poly_data], 'p-', 
        color='purple', label='NGRC Polynomial', markersize=10, linewidth=2)

# Fourier (best sigma)
fourier_data = [(results[k]['dim'], results[k]['psnr']) for k in results 
                if 'fourier' in k and '_s10' in k]  # sigma=10
fourier_data.sort()
ax.plot([d[0] for d in fourier_data], [d[1] for d in fourier_data], 's-', 
        color='blue', label='Fourier (σ=10)', markersize=10, linewidth=2)

# Random tanh
random_data = [(results[k]['dim'], results[k]['psnr']) for k in results 
               if 'random_tanh' in k]
random_data.sort()
ax.plot([d[0] for d in random_data], [d[1] for d in random_data], 'o-', 
        color='orange', label='Random tanh', markersize=10, linewidth=2)

# Random sin  
random_sin = [(results[k]['dim'], results[k]['psnr']) for k in results 
              if 'random_sin' in k]
random_sin.sort()
ax.plot([d[0] for d in random_sin], [d[1] for d in random_sin], '^-', 
        color='red', label='Random sin', markersize=10, linewidth=2)

# Traditional reservoir
res_data = [(results[k]['dim'], results[k]['psnr']) for k in results if 'reservoir' in k]
for d, p in res_data:
    ax.scatter([d], [p], color='green', s=150, marker='*', zorder=5)
ax.scatter([], [], color='green', s=150, marker='*', label='Traditional Reservoir')

ax.set_xlabel('Feature Dimension', fontsize=12)
ax.set_ylabel('PSNR (dB)', fontsize=12)
ax.set_title('PSNR vs Feature Dimension', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_xscale('log')

plt.tight_layout()
plt.savefig('ngrc_scaling.png', dpi=150, bbox_inches='tight')
plt.show()

## Key Insights

In [ ]:
print("\n" + "=" * 70)
print("KEY INSIGHTS: NGRC vs TRADITIONAL APPROACHES")
print("=" * 70)

print("""
1. NGRC PRINCIPLE APPLIED TO INR
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   NGRC replaces recurrent dynamics with explicit polynomial features.
   For spatial INR: polynomial features of (x,y) coordinates.
   
   This is essentially POLYNOMIAL REGRESSION on coordinates.

2. WHY FOURIER STILL WINS
   ━━━━━━━━━━━━━━━━━━━━━━
   - Natural images are BANDLIMITED (smooth, not polynomial)
   - Fourier basis matches image statistics
   - Polynomials diverge at boundaries, oscillate badly
   - sin/cos are globally smooth and periodic

3. WHAT NGRC TEACHES US
   ━━━━━━━━━━━━━━━━━━━━
   - Recurrence is NOT necessary for good features
   - The right BASIS FUNCTION matters more than architecture
   - For temporal data: time-delay polynomials work
   - For spatial data: Fourier (frequency) basis works

4. RANDOM sin(Wx+b) ≈ FOURIER
   ━━━━━━━━━━━━━━━━━━━━━━━━━━
   Random sinusoidal projection approximates Fourier features!
   This connects NGRC random projection to Fourier methods.

5. RESERVOIR'S REAL VALUE
   ━━━━━━━━━━━━━━━━━━━━━━
   - NOT in basis function quality
   - IN temporal memory and dynamics
   - For static INR: use Fourier or NGRC polynomial
   - For temporal: use ESN or NGRC time-delay features
""")

## Bonus: Hybrid NGRC + Fourier

In [ ]:
print("\n" + "=" * 60)
print("HYBRID: NGRC POLYNOMIAL + FOURIER")
print("=" * 60)

# Combine polynomial and Fourier features
for poly_deg in [3, 5]:
    for fourier_n in [256, 512]:
        H_poly_train = ngrc_polynomial_features_fast(train_data[0], poly_deg)
        H_poly_test = ngrc_polynomial_features_fast(test_data[0], poly_deg)
        
        H_fourier_train = fourier_features(train_data[0], fourier_n, sigma=10)
        H_fourier_test = fourier_features(test_data[0], fourier_n, sigma=10)
        
        H_train = np.hstack([H_poly_train, H_fourier_train])
        H_test = np.hstack([H_poly_test, H_fourier_test])
        
        pred, mse, psnr, W = ridge_regression(H_train, train_data[1], H_test, test_data[1])
        name = f'hybrid_poly{poly_deg}_fourier{fourier_n}'
        results[name] = {'pred': pred, 'mse': mse, 'psnr': psnr, 'dim': H_train.shape[1]}
        print(f"  Poly d={poly_deg} + Fourier n={fourier_n}: PSNR = {psnr:.2f} dB (dim={H_train.shape[1]})")

In [ ]:
# Final summary
print("\n" + "=" * 70)
print("FINAL RANKING (Top 10)")
print("=" * 70)

sorted_all = sorted(results.items(), key=lambda x: x[1]['psnr'], reverse=True)
print(f"{'Rank':<6} {'Method':<40} {'PSNR (dB)':<12} {'Dim':<8}")
print("-" * 66)
for i, (name, r) in enumerate(sorted_all[:10]):
    print(f"{i+1:<6} {name:<40} {r['psnr']:<12.2f} {r['dim']:<8}")